In [ ]:
pip install pytorch-adapt

In [ ]:
import sys

sys.path.insert(0, "../../src")

### Create models, optimizers, data etc.

In [ ]:
import torch

from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.hooks import validate_hook

G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Sequential(torch.nn.Linear(100, 1), torch.nn.Flatten(start_dim=0))

models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.456}))
optimizers.create_with(models)
opts = list(optimizers.values())

data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
    "src_labels": torch.randint(0, 10, size=(32,)),
    "src_domain": torch.zeros(32),
    "target_domain": torch.zeros(32),
    "src_sample_idx": torch.randint(0, 10000, size=(32,)),
    "target_sample_idx": torch.randint(0, 10000, size=(32,)),
}

### DANN

In [ ]:
from pytorch_adapt.hooks import DANNHook

hook = DANNHook(opts)
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **data})

### DANN + MCC + ATDOC

In [ ]:
from pytorch_adapt.hooks import ATDOCHook, MCCHook

mcc = MCCHook()
atdoc = ATDOCHook(dataset_size=10000, feature_dim=100, num_classes=10)

hook = DANNHook(opts, post_g=[mcc, atdoc])
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **data})

### CDAN

In [ ]:
from pytorch_adapt.hooks import CDANHook
from pytorch_adapt.layers import RandomizedDotProduct
from pytorch_adapt.utils import common_functions as c_f

d_opts = c_f.extract(optimizers, ["D"])
g_opts = c_f.extract(optimizers, ["G", "C"])
misc = {"feature_combiner": RandomizedDotProduct([100, 10], 100)}

hook = CDANHook(d_opts=d_opts, g_opts=g_opts)
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **misc, **data})

### CDAN + VAT

In [ ]:
from pytorch_adapt.hooks import VATHook

misc["combined_model"] = torch.nn.Sequential(G, C)
hook = CDANHook(d_opts=d_opts, g_opts=g_opts, post_g=[VATHook()])
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **misc, **data})

### MCD

In [ ]:
from pytorch_adapt.hooks import MCDHook
from pytorch_adapt.layers import MultipleModels

C = torch.nn.Linear(100, 10)
C1, C2 = C, c_f.reinit(C)
C = MultipleModels(C1, C2)
models["C"] = C

g_opts = c_f.extract(optimizers, ["G"])
c_opts = c_f.extract(optimizers, ["C"])

hook = MCDHook(g_opts=g_opts, c_opts=c_opts)
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **data})

### MCD + AFN + MMD

In [ ]:
from pytorch_adapt.hooks import AFNHook, AlignerHook

hook = MCDHook(g_opts=g_opts, c_opts=c_opts, post_x=[AFNHook()], post_z=[AlignerHook()])
validate_hook(hook, list(data.keys()))
losses, outputs = hook({}, {**models, **data})